In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import StandardScaler

In [2]:
train=pd.read_csv('/Users/taewon/Documents/금융 빅데이터/Project_2/코딩/상장기업/sampling data/Undersampling_0.33_train.csv',index_col=False, encoding='euc-kr')
test=pd.read_csv('/Users/taewon/Documents/금융 빅데이터/Project_2/코딩/상장기업/sampling data/Undersampling_0.33_test.csv',index_col=False,encoding='euc-kr')

In [3]:
X_train_int=train[['부가가치율', '자기자본구성비율', '총자산대비영업현금흐름', '자기자본순이익률', '총자본순이익률', '총자본회전률',
       'PBR', '이윤분배율', '자기자본증가율', '차입금의존도', 'log자산총계', 'PER', '연구개발비대비매출액',
       '유동자산회전률']]

X_test_int=test[['부가가치율', '자기자본구성비율', '총자산대비영업현금흐름', '자기자본순이익률', '총자본순이익률', '총자본회전률',
       'PBR', '이윤분배율', '자기자본증가율', '차입금의존도', 'log자산총계', 'PER', '연구개발비대비매출액',
       '유동자산회전률']]

In [4]:
X_train = train.drop('t-1감사의견코드',axis=1)
y_train = train[['t-1감사의견코드']]

X_test = test.drop('t-1감사의견코드',axis=1)
y_test = test[['t-1감사의견코드']]

In [5]:
X_train_sc = X_train
X_test_sc = X_test

In [6]:
X_train_sum =X_train_sc
X_test_sum =X_test_sc

In [7]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from xgboost import XGBClassifier
import numpy as np

def perform_xgb_grid_search(X_train, y_train, k_fold=5):
    # Stratified k-fold cross validation setup
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # XGBoost hyperparameter candidate list setup
    param_grid = {
        'max_depth': [3, 5, 7],
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'objective': ['binary:logistic'],
        'random_state': [0]
    }

    # Initialize an XGBoost classifier
    model = XGBClassifier(use_label_encoder=False)

    # Grid search setup
    grid_search = GridSearchCV(model, param_grid, scoring='f1', cv=cv, verbose=1, n_jobs=-1)

    # Fit the model and tune
    grid_search.fit(X_train, y_train)

    # Print the best hyperparameters
    print("Best Hyperparameters:", grid_search.best_params_)

    # Calculate the average evaluation metric
    mean_f1_score = np.mean(grid_search.cv_results_['mean_test_score'])
    print("Mean F1 Score:", mean_f1_score)

    return grid_search.best_params_, mean_f1_score


In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from xgboost import XGBClassifier
import numpy as np

def evaluate_xgb_with_best_params(X_train, y_train, X_test, y_test, best_params, k_fold=5):
    # Stratified k-fold cross validation setup
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # Initialize an XGBoost model
    model = XGBClassifier(**best_params, use_label_encoder=False)

    # Lists to save the evaluation metrics for each fold
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []
    confusion_matrix_list = []

    best_f1_score = 0
    best_model = None

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_test_fold, y_test_fold = X_train.iloc[test_idx], y_train.iloc[test_idx]

        # Fit the model
        model.fit(X_train_fold, y_train_fold)

        # Get the predicted probabilities on the test data
        probabilities = model.predict_proba(X_test_fold)

        # Adjust the predicted classes with a threshold of 0.5
        threshold = 0.5
        predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)

        # Calculate the evaluation metrics
        accuracy = accuracy_score(y_test_fold, predicted_classes)
        precision = precision_score(y_test_fold, predicted_classes)
        recall = recall_score(y_test_fold, predicted_classes)
        f1 = f1_score(y_test_fold, predicted_classes)
        conf_matrix = confusion_matrix(y_test_fold, predicted_classes)

        # Append the evaluation metrics for each fold to the lists
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1)
        confusion_matrix_list.append(conf_matrix)

        print(f"Fold {fold_idx}")
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 score: {f1}")
        print("Confusion Matrix:")
        print(conf_matrix)
        print("------------------------------")

        # Save the model with the best F1 score
        if f1 > best_f1_score:
            best_f1_score = f1
            best_model = model

    # Perform the final prediction with the model with the best F1 score
    probabilities_final = best_model.predict_proba(X_test)
    y_pred_final = (probabilities_final[:, 1] > threshold).astype(np.int)

    # Calculate the evaluation metrics
    accuracy_final = accuracy_score(y_test, y_pred_final)
    precision_final = precision_score(y_test, y_pred_final)
    recall_final = recall_score(y_test, y_pred_final)
    f1_final = f1_score(y_test, y_pred_final)
    conf_matrix_final = confusion_matrix(y_test, y_pred_final)

    print("Final Test Results")
    print(f"Accuracy: {accuracy_final}")
    print(f"Precision: {precision_final}")
    print(f"Recall: {recall_final}")
    print(f"F1 score: {f1_final}")
    print("Confusion Matrix:")
    print(conf_matrix_final)

    return accuracy_list, precision_list, recall_list, f1_score_list


In [9]:
best_params, mean_f1_score = perform_xgb_grid_search(X_train, y_train, k_fold=5)
evaluate_xgb_with_best_params(X_train_int, y_train, X_test_int, y_test, best_params, k_fold=5)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


/Users/taewon/Library/Python/3.9/lib/python/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/taewon/Library/Python/3.9/lib/python/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 300, 'objective': 'binary:logistic', 'random_state': 0}
Mean F1 Score: 0.6073448494037083
Fold 1
Accuracy: 0.8493150684931506
Precision: 0.7692307692307693
Recall: 0.5555555555555556
F1 score: 0.6451612903225806
Confusion Matrix:
[[52  3]
 [ 8 10]]
------------------------------


/Users/taewon/Library/Python/3.9/lib/python/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/var/folders/28/yz261nq91vq_lj47hynw_10r0000gn/T/ipykernel_70623/2210109296.py:35: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)
/var/folders/28/yz261nq91vq_lj47hynw_10r0000gn/T/ipykernel_70623/2210109296.py:35: DeprecationWarning: `np.int` is a deprecated alias for the builtin `

Fold 2
Accuracy: 0.7945205479452054
Precision: 0.5714285714285714
Recall: 0.6666666666666666
F1 score: 0.6153846153846153
Confusion Matrix:
[[46  9]
 [ 6 12]]
------------------------------
Fold 3
Accuracy: 0.7777777777777778
Precision: 0.6
Recall: 0.3333333333333333
F1 score: 0.42857142857142855
Confusion Matrix:
[[50  4]
 [12  6]]
------------------------------
Fold 4
Accuracy: 0.9027777777777778
Precision: 0.8235294117647058
Recall: 0.7777777777777778
F1 score: 0.7999999999999999
Confusion Matrix:
[[51  3]
 [ 4 14]]
------------------------------
Fold 5
Accuracy: 0.8333333333333334
Precision: 0.8
Recall: 0.4444444444444444
F1 score: 0.5714285714285714
Confusion Matrix:
[[52  2]
 [10  8]]
------------------------------
Final Test Results
Accuracy: 0.8055555555555556
Precision: 0.6363636363636364
Recall: 0.5185185185185185
F1 score: 0.5714285714285714
Confusion Matrix:
[[73  8]
 [13 14]]


/var/folders/28/yz261nq91vq_lj47hynw_10r0000gn/T/ipykernel_70623/2210109296.py:35: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)
/var/folders/28/yz261nq91vq_lj47hynw_10r0000gn/T/ipykernel_70623/2210109296.py:35: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precisio

([0.8493150684931506,
  0.7945205479452054,
  0.7777777777777778,
  0.9027777777777778,
  0.8333333333333334],
 [0.7692307692307693, 0.5714285714285714, 0.6, 0.8235294117647058, 0.8],
 [0.5555555555555556,
  0.6666666666666666,
  0.3333333333333333,
  0.7777777777777778,
  0.4444444444444444],
 [0.6451612903225806,
  0.6153846153846153,
  0.42857142857142855,
  0.7999999999999999,
  0.5714285714285714])